# Geopandas and Spatial Databases

In this lecture we are going to interact with a spatial database through Python using geopandas

## Connecting to Spatial Database

First let us make the necessary imports

In [1]:
import geopandas as gpd
import pandas as pd
import psycopg2 #this is for connecting to the postgresql database

Create a connection object

In [2]:
conn = psycopg2.connect("dbname=nyc user=geospatial password=geospatial2023 host=ghhlibrary")

Now we can use the connection object for retreiving data from the spatial database. 

**Note: If your query is returning geometry then use geopandas and if your query is not returning any geometry columns use pandas**

Let us first try a query that returns some geometry data. 

We want to return all the schools with in the 'Harlem' neighborhood in 'Manhattan'. How will a raw query look like

```sql
select s.* from nyc_schools s,nyc_neighborhoods n where n.name='Harlem' and n.boroname = 'Manhattan' and st_within(s.geom,n.geom)
```

As you can see the query returns a geometry column (geom). So let us use postgis to read the data from the table

In [3]:
schoolsHarlem = gpd.read_postgis("select s.* from nyc_schools s,nyc_neighborhoods n where n.name='Harlem' and n.boroname = 'Manhattan' and st_within(s.geom,n.geom)",conn)

E:\sbd2\lib\site-packages\geopandas\io\sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [ ]:
schoolsHarlem

In [5]:
type(schoolsHarlem)

geopandas.geodataframe.GeoDataFrame

As you can see, the read_postgis method returns a geodataframe object as the result of the query. You can see the geometry column at the end

In [6]:
schoolsHarlem.crs

<Derived Projected CRS: EPSG:26918>
Name: NAD83 / UTM zone 18N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: North America - between 78°W and 72°W - onshore and offshore. Canada - Nunavut; Ontario; Quebec. United States (USA) - Connecticut; Delaware; Maryland; Massachusetts; New Hampshire; New Jersey; New York; North Carolina; Pennsylvania; Virginia; Vermont.
- bounds: (-78.0, 28.28, -72.0, 84.0)
Coordinate Operation:
- name: UTM zone 18N
- method: Transverse Mercator
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

Now let us try interactive mapping for the first time. First let us create an interactive map using ipyleaflet

In [28]:
from ipyleaflet import Map, GeoData
mapView = Map(center=(40.712778, -74.006111), zoom=15)
mapView

Map(center=[40.712778, -74.006111], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

Yay!!!!!! we have created an interactive map. 

Now let us try to add our geodataframe to the map

Make sure the dataframe is in EPSG:4326

In [29]:
schoolsHarlem = schoolsHarlem.to_crs('EPSG:4326')

In [30]:
geo_data = GeoData(geo_dataframe = schoolsHarlem)
mapView.add_layer(geo_data)

If you zoom out and pan the map you should be able to see the locations. 

So let us try another query. This time we are going to find out those schools that are with in 20 meters of a shooting incident.

Our query

```sql
select distinct s.gid,s.geom from nyc_schools s,nypd_shooting n where st_dwithin(s.geom,n.geom,20)
```

In [40]:
schoolsShooting20m = gpd.read_postgis("select distinct s.gid,s.geom from nyc_schools s,nypd_shooting n where st_dwithin(s.geom,n.geom,20)",conn)

E:\sbd2\lib\site-packages\geopandas\io\sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [41]:
schoolsShooting20m

,gid,geom
0,1090,POINT (590978.473 4517825.350)
1,1176,POINT (593297.342 4523317.062)
2,1183,POINT (591606.812 4522050.170)
3,457,POINT (589545.364 4516541.662)


In [42]:
mapView = Map(center=(40.712778, -74.006111), zoom=15)
mapView

Map(center=[40.712778, -74.006111], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [43]:
geo_data = GeoData(geo_dataframe = schoolsShooting20m.to_crs('EPSG:4326'))
mapView.add_layer(geo_data)

Now let us try a query with polygons. We need to find out taxi zones with more than 20 schools in it and then display the polygons. This is a bit more complex query

```sql
select t.locationid,zone,count(s.gid) as total_schools,t.geom from taxi_zones t,nyc_schools s where
st_contains(t.geom,s.geom) group by t.locationid having count(s.gid)>20
```

Make a notice of the having clause, it helps to filter out the group by results. Now let us run this with geopandas

In [46]:
taxiMore20Schools = gpd.read_postgis("select t.locationid,zone,count(s.gid) as total_schools,t.geom from taxi_zones t,nyc_schools s where st_contains(t.geom,s.geom) group by t.locationid having count(s.gid)>20",conn)

E:\sbd2\lib\site-packages\geopandas\io\sql.py:154: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [47]:
taxiMore20Schools

,locationid,zone,total_schools,geom
0,42,Central Harlem North,25,"MULTIPOLYGON (((589844.118 4521120.403, 589836..."
1,61,Crown Heights North,21,"MULTIPOLYGON (((590489.482 4504167.728, 590503..."
2,35,Brownsville,30,"MULTIPOLYGON (((592894.323 4501831.339, 592929..."
3,75,East Harlem South,28,"MULTIPOLYGON (((589339.992 4516267.612, 589542..."
4,47,Claremont/Bathgate,26,"MULTIPOLYGON (((593483.050 4523154.756, 593378..."
5,76,East New York,24,"MULTIPOLYGON (((594595.879 4503746.008, 594598..."
6,69,East Concourse/Concourse Village,27,"MULTIPOLYGON (((591760.301 4520058.958, 591803..."
7,17,Bedford,22,"MULTIPOLYGON (((589290.612 4506150.301, 589364..."
8,74,East Harlem North,22,"MULTIPOLYGON (((589917.530 4518935.414, 589916..."
9,225,Stuyvesant Heights,28,"MULTIPOLYGON (((591481.459 4504246.243, 591494..."


In [48]:
mapView = Map(center=(40.712778, -74.006111), zoom=15)
mapView

Map(center=[40.712778, -74.006111], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [49]:
geo_data = GeoData(geo_dataframe = taxiMore20Schools.to_crs('EPSG:4326'))
mapView.add_layer(geo_data)

Now let us try practice some more spatial queries

1. Find total homicide per population for all census blocks in nyc. 

2. Display top 5 neighborhoods with most number of shootings. 

3. Calculate total population of all counties in nyc (use nyc census blocks to get population).

4. Calculate total population of all neighborhoods using the census blocks data. 

5. Find all subway stations in Harlem neighbrohood in Manhattan. 